# Генерация и анализ CAD-моделей мебели различных стилей на основе текстовых описаний

--------------

## Варианты

---------------

1. Работа с Fusion360. Скачиваем много разнообразных моделей с grabcad.com в формате .f3d. Скачиваем официальную trial-версию Fusion через пень-колоду. Проводим параметризацию 3D-модели через API и VS Code - но там есть загвоздка, что названия операций придется все равно прописывать вручную через "Change Parameters", потому что встроенный инструмент design.allParameters позволяет вытащить только параметрическое описание, соответственно в 'name' словаря будет храниться его параметрическое значение вроде 'd1', 'd2' и т.д, а вот что такое 'd1' и 'd2' нам поможет понять 'Change Parameters'. К примеру в той модели, которую я смотрю, 'd1' - это 'LinearDimension' в 'Sketch', а 'd3' - 'AlongDistance' в 'Extrude'. Также API Фьюжена не дает возможность вытащить последовательность формирования объекта, вроде "Sketch1, Extrude1, Sketch2, Extrude2" и т.д. То есть тоже придется этот момент прописывать в промпт вручную. Промпт выдает код под CadQuery, мы его еще очень долго редактируем и получаем на выходе какую-то модель, которую также анализируем на сходство с оригиналом, совпадение размеров, ошибки и т.д. Визуализация прямо в ноутбуке.

2. Работа с FreeCAD. Также находим множество моделей (кстати, для FreeCAD моделей мебели куда меньше, чем для Fusion), открываем и анализируем их геометрию в программе - с помощью инструмента Measurement делаем замеры, формируем вручную подробный промпт, редактируем код, запускаем и дальше как в конце первого пункта.

3. Студенты сами придумывают различные варианты мебели, можно из гугла, можно из головы, пытаясь идеально воплотить задумку. То есть они составят максимально подробный промпт быстрее, чем в двух пунктах, потому что немало времени в первых двух пунктах займет анализ модели. Значит смогут больше поработать с библиотекой, и сделать куда более качественный и доступный код, причем наверняка получившаяся модель будет ближе к реальности, чем в первых двух пунктах. Дальше все как в конце первых двух пунктов.

От себя: у меня пока что вообще не вышло ничего адекватно похожего на реальную модель. Только на toy-examples


## Код под Fusion: (можно не запускать, он только в среде фьюжэна запустится)

-----------------

In [68]:
import adsk.core
import adsk.fusion
import traceback

app = adsk.core.Application.get()
ui = app.userInterface

def get_model_parameters():
    design = adsk.fusion.Design.cast(app.activeProduct)
    if not design:
        ui.messageBox("Активный документ не является дизайном Fusion 360.")
        return None

    root_comp = design.rootComponent

    params = []
    for param in design.allParameters:
        params.append({
            "name": param.name,
            "value": param.value,
            "unit": param.unit
        })

    features = root_comp.features
    operations = []

    for extrude in features.extrudeFeatures:
        operations.append({"type": "Extrude", "name": extrude.name})

    for fillet in features.filletFeatures:
        operations.append({"type": "Fillet", "name": fillet.name})

    for hole in features.holeFeatures:
        operations.append({"type": "Hole", "name": hole.name})

    for revolve in features.revolveFeatures:
        operations.append({"type": "Revolve", "name": revolve.name})

    return {"parameters": params, "operations": operations}


def run(_context: str):
    try:
        result = get_model_parameters()
        if result:
            ui.messageBox(f'Параметры и операции модели:\n{result}')
    except:
        app.log(f'Ошибка:\n{traceback.format_exc()}')

ModuleNotFoundError: No module named 'adsk'

-------------

# Первым делом устанавливаем Anaconda и прикручиваем все, что связано с CADQuery. Последовательность та же, что в репе.

-----------

## 3D Модели - студенты ищут сами или же я скачаю с grabcad и закину всем на гитхаб? Пока не продумал, как сделать так, чтобы за определенным студентом закреплялась определенная деталь и ее автоматически никто в будущем не мог взять, чтобы дважды с одной и той же никто не работал.

## 1. GigaChat API + генерация текстовых описаний мебели

In [23]:
pip install gigachat

  Using cached gigachat-0.1.38-py3-none-any.whl.metadata (14 kB)
Using cached gigachat-0.1.38-py3-none-any.whl (62 kB)
   ---------------------------------------- 0.0/76.4 kB ? eta -:--:--
   ----- ---------------------------------- 10.2/76.4 kB ? eta -:--:--
   ---------------- ----------------------- 30.7/76.4 kB 640.0 kB/s eta 0:00:01
   ---------------- ----------------------- 30.7/76.4 kB 640.0 kB/s eta 0:00:01
   ---------------------------------------- 76.4/76.4 kB 469.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/78.6 kB ? eta -:--:--
   ----- ---------------------------------- 10.2/78.6 kB ? eta -:--:--
   ----- ---------------------------------- 10.2/78.6 kB ? eta -:--:--
   --------------- ------------------------ 30.7/78.6 kB 217.9 kB/s eta 0:00:01
   -------------------- ------------------- 41.0/78.6 kB 217.9 kB/s eta 0:00:01
   -------------------- ------------------- 41.0/78.6 kB 217.9 kB/s eta 0:00:01
   ---------------------------------------- 78.6


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Тут только на свой Authorization Key поменять:

In [56]:
from gigachat import GigaChat

with GigaChat(credentials="YzdhNTY1OTMtOGJlZi00MmUzLWFlYzEtZWI5YTQ5ZmI3NjZmOjA2OGY0YzkxLTVmODYtNGI3OC04ZjI3LTllZTIyY2VkYzJiZQ==", verify_ssl_certs=False) as giga:
    response = giga.chat("Сформируй код для библиотеки Cadquery в Python, который позволит сгенерировать модель со следующими параметрами и операциями модели: Параметры и операции модели: {'parameters': [{'name': 'd1', 'value': 2.5, 'unit': 'mm'}, {'name': 'd2', 'value': 240.0, 'unit': 'mm'}, {'name': 'd3', 'value': 100.0, 'unit': 'mm'}, {'name': 'd4', 'value': 0.0, 'unit': 'deg'}, {'name': 'd5', 'value': 15.0, 'unit': 'mm'}, {'name': 'd6', 'value': 6.5, 'unit': 'mm'}, {'name': 'd7', 'value': 90.0, 'unit': 'mm'}, {'name': 'd8', 'value': 6.5, 'unit': 'mm'}, {'name': 'd9', 'value': 0.0, 'unit': 'deg'}, {'name': 'd10', 'value': 5.0, 'unit': 'mm'}, {'name': 'd11', 'value': 35.0, 'unit': 'mm'}, {'name': 'd12', 'value': 6.5, 'unit': 'mm'}, {'name': 'd13', 'value': 6.5, 'unit': 'mm'}, {'name': 'd14', 'value': 6.5, 'unit': 'mm'}, {'name': 'd15', 'value': 0.0, 'unit': 'deg'}, {'name': 'd16', 'value': 60.0, 'unit': 'mm'}, {'name': 'd17', 'value': 0.0, 'unit': 'deg'}, {'name': 'd18', 'value': 6.5, 'unit': 'mm'}, {'name': 'd19', 'value': 0.0, 'unit': 'deg'}, {'name': 'd20', 'value': 5.0, 'unit': 'mm'}, {'name': 'd21', 'value': 0.0, 'unit': 'mm'}, {'name': 'd22', 'value': 0.0, 'unit': 'deg'}, {'name': 'd23', 'value': 1.8, 'unit': 'mm'}, {'name': 'd24', 'value': 1.8, 'unit': 'mm'}, {'name': 'd25', 'value': 1.8, 'unit': 'mm'}, {'name': 'd26', 'value': 1.8, 'unit': 'mm'}, {'name': 'd27', 'value': -2.5, 'unit': 'mm'}, {'name': 'd28', 'value': 0.0, 'unit': 'deg'}, {'name': 'd29', 'value': -2.5, 'unit': 'mm'}, {'name': 'd30', 'value': 0.0, 'unit': 'deg'}, {'name': 'd31', 'value': 0.30000000000000004, 'unit': 'mm'}, {'name': 'd32', 'value': 0.30000000000000004, 'unit': 'mm'}, {'name': 'd33', 'value': 5.0, 'unit': 'mm'}, {'name': 'd34', 'value': 5.0, 'unit': 'mm'}, {'name': 'd35', 'value': -6.5, 'unit': 'mm'}, {'name': 'd36', 'value': 0.0, 'unit': 'deg'}, {'name': 'd37', 'value': 1.6, 'unit': 'mm'}, {'name': 'd38', 'value': 0.4, 'unit': 'mm'}, {'name': 'd39', 'value': 2.059488517353309, 'unit': 'deg'}, {'name': 'd40', 'value': 1.6, 'unit': 'mm'}, {'name': 'd41', 'value': 0.4, 'unit': 'mm'}, {'name': 'd42', 'value': 2.059488517353309, 'unit': 'deg'}, {'name': 'd43', 'value': 1.6, 'unit': 'mm'}, {'name': 'd44', 'value': 0.4, 'unit': 'mm'}, {'name': 'd45', 'value': 2.059488517353309, 'unit': 'deg'}, {'name': 'd46', 'value': 1.6, 'unit': 'mm'}, {'name': 'd47', 'value': 0.4, 'unit': 'mm'}, {'name': 'd48', 'value': 2.059488517353309, 'unit': 'deg'}], 'operations': [{'type': 'Extrude', 'name': 'Extrude1'}, {'type': 'Extrude', 'name': 'Extrude2'}, {'type': 'Extrude', 'name': 'Extrude3'}, {'type': 'Extrude', 'name': 'Extrude4'}, {'type': 'Extrude', 'name': 'Extrude5'}, {'type': 'Extrude', 'name': 'Extrude6'}, {'type': 'Extrude', 'name': 'Extrude7'}, {'type': 'Extrude', 'name': 'Extrude8'}, {'type': 'Extrude', 'name': 'Extrude9'}, {'type': 'Hole', 'name': 'Hole1'}, {'type': 'Hole', 'name': 'Hole2'}, {'type': 'Hole', 'name': 'Hole3'}, {'type': 'Hole', 'name': 'Hole4'}]}")
    print(response.choices[0].message.content)

Для того чтобы создать код в библиотеке `Cadquery` на основе предоставленных параметров и операций, нужно пройтись по каждому элементу в словаре `parameters` и затем воспользоваться этими данными для создания объектов в Cadquery.

Таким образом, сначала создадим все необходимые параметры и затем используем их для создания различных частей модели. Например, мы можем создать боковые стенки, основания, крышки и т.д., используя операции типа `Extrude` и `Hole`.

Вот пример кода на Python, который реализует вышеописанное:

```python
import cadquery
from math import pi

# Параметры модели
params = {
    'd1': 2.5,                   # Толщина стенок
    'd2': 240.0,                 # Длина
    'd3': 100.0,                 # Ширина
    'd4': 0.0,                   # Угол поворота (не используется)
    'd5': 15.0,                  # Диаметр отверстий
    'd6': 6.5,                   # Диаметр крепежных отверстий
    'd7': 90.0,                  # Высота корпуса
    'd8': 6.5,                   

## 2. Генерация CAD-модели

Студенты преобразуют данные от LLM в программный код и формируют 3D-модели, параллельно редактируя код для достижения наиболее корректного результата:

In [1]:
import cadquery as cq
from jupyter_cadquery import show

Overwriting auto display for cadquery Workplane and Shape


In [2]:
# Параметры модели
params = {
    'd1': 2.5,                   # Толщина стенок
    'd2': 240.0,                 # Длина
    'd3': 100.0,                 # Ширина
    'd4': 0.0,                   # Угол поворота (не используется)
    'd5': 15.0,                  # Диаметр отверстий
    'd6': 6.5,                   # Диаметр крепежных отверстий
    'd7': 90.0,                  # Высота корпуса
    'd8': 6.5,                   # Высота ребер жесткости
    'd9': 0.0,                   # Угол поворота ребер жесткости
    'd10': 5.0,                  # Расстояние между отверстиями
    'd11': 35.0,                 # Толщина крышек
    'd12': 6.5,                  # Толщина ребер жесткости
    'd13': 6.5,                  # Толщина днища
    'd14': 6.5,                  # Толщина верхней части
    'd15': 0.0,                  # Угол поворота крышек
    'd16': 60.0,                 # Высота боковых стенок
    'd17': 0.0,                  # Угол поворота боковых стенок
    'd18': 6.5,                  # Толщина верхних/нижних граней
    'd19': 0.0,                  # Угол поворота верхних/нижних граней
    'd20': 5.0,                  # Расстояние между отверстиями на крышках
    'd21': 0.0,                  # Угол поворота крышек
    'd22': 0.0,                  # Угол поворота осей вращения
    'd23': 1.8,                  # Толщина осей вращения
    'd24': 1.8,                  # Диаметр осей вращения
    'd25': 1.8,                  # Количество осей вращения
    'd26': 1.8,                  # Дополнительная толщина крышек
    'd27': -2.5,                 # Глубина прорезей
    'd28': 0.0,                  # Угол поворота прорезей
    'd29': -2.5,                 # Вторая глубина прорезей
    'd30': 0.0,                  # Угол поворота второй глубины прорезей
    'd31': 0.30000000000000004,  # Толщина крышек сверху
    'd32': 0.30000000000000004,  # Толщина крышек снизу
    'd33': 5.0,                  # Размер опорной пластины
    'd34': 5.0,                  # Размер опорной пластины
    'd35': -6.5,                 # Глубина нижней части
    'd36': 0.0,                  # Угол поворота нижней части
    'd37': 1.6,                  # Толщина оси вращения
    'd38': 0.4,                  # Толщина шайбы
    'd39': 2.059488517353309,   # Угол наклона оси вращения
    'd40': 1.6,                  # Толщина шайбы
    'd41': 0.4,                  # Толщина шайбы
    'd42': 2.059488517353309,   # Угол наклона оси вращения
    'd43': 1.6,                  # Толщина шайбы
    'd44': 0.4,                  # Толщина шайбы
    'd45': 2.059488517353309,   # Угол наклона оси вращения
    'd46': 1.6,                  # Толщина шайбы
    'd47': 0.4,                  # Толщина шайбы
    'd48': 2.059488517353309,   # Угол наклона оси вращения
}

# Операции модели
operations = [
    {"type": "Extrude", "name": "Extrude1", "depth": params['d7']},
    {"type": "Extrude", "name": "Extrude2", "depth": params['d17']},
    {"type": "Extrude", "name": "Extrude3", "depth": params['d19']},
    {"type": "Extrude", "name": "Extrude4", "depth": params['d18']},
    {"type": "Extrude", "name": "Extrude5", "depth": params['d15']},
    {"type": "Extrude", "name": "Extrude6", "depth": params['d14']},
    {"type": "Extrude", "name": "Extrude7", "depth": params['d16']},
    {"type": "Extrude", "name": "Extrude8", "depth": params['d18']},
    {"type": "Extrude", "name": "Extrude9", "depth": params['d19']},
    {"type": "Hole", "name": "Hole1", "diameter": params['d5'], "center": (params['d2'], params['d3'])},
    {"type": "Hole", "name": "Hole2", "diameter": params['d6'], "center": (params['d2'], params['d3'])},
    {"type": "Hole", "name": "Hole3", "diameter": params['d6'], "center": (params['d2'] + params['d16'], params['d3'])},
    {"type": "Hole", "name": "Hole4", "diameter": params['d6'], "center": (params['d2'] + params['d16'], params['d3'] + params['d16'])},
]

# Создание корпуса
cq_body = cq.Workplane("XY").box(params['d2'], params['d3'], params['d7'])

# Применяем операции
for operation in operations:
    if operation["type"] == "Extrude":
        cq_body = cq_body.extrude(operation["depth"])  # Используем метод extrude
    elif operation["type"] == "Hole":
        cq_body = cq_body.hole(operation["diameter"]).translate(operation["center"])  # Применяем отверстие

# Экспорт модели
cq_body

ValueError: No pending wires present

In [3]:
# Параметры
d1 = 55.88  # in
d2 = 15.24  # in
d3 = 10.16  # in

# Создаём начальную экструзию (прямоугольник)
part = cq.Workplane("XY").rect(d1, d2).extrude(d3)  # Прямоугольник с экструзией

# Добавление вращения (например, вокруг оси Z)
part = part.faces(">Z").workplane().polyline([(0, 0), (d1, 0), (d1, d2), (0, d2), (0, 0)]).close().revolve(360, (0, 0, 0), (0, 0, 1))

# Отобразим результат
part

CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…

## 3. Анализ ошибок и выявление закономерностей

Студент после получения моделей изделий должен подготовить небольшой отчет (можно прям в своем ноутбуке), включающий в себя:

1. Типы ошибок при формировании модели (ПРИМЕР: ошибки геометрии - пересечения объектов, отсутствующие или неправильные соединения элементов и т.д; ошибки с размерами; ошибки в структуре модели - недостающие элементы, лишние элементы и др.)

2. Факторы, влияющие на качество генерации (ПРИМЕР: полнота текстового описания, сложность описания (может, какие-то конкретные слова ломали LLM), выбранный стиль мебели, детализация мебели и т.д.)

3. Какие улучшения можно предложить для получения более корректного решения (так мы потом рекомендации составим)

## 4. Визуализация и отчет

В продолжение формирования отчета, в отчете также должны быть представлены:

1. Скриншоты оригинальных моделей, вставленные в Jupyter Notebook через plt.imshow() и получившаяся модель, которая будет показана через jupyter_cadquery.show()

2. График (возможно, гистограмма) зависимости успешных и неуспешных генерацией для разных стилей мебели

(3. В идеале, хотелось бы видеть красивую таблицу: номер детали, ее текстовое описание, оригинальная модель (скриншот), полученная модель (скриншот), ошибки, причины, варианты исправлений)